In [1]:
import petl as etl
from pathlib import Path
from zipfile import ZipFile

Set up some sources

In [2]:
ROOT = Path('../../..')
OUT_PATH = ROOT / 'data/processed/areas/'

Find the zip file. NB - this is downloaded in the education sources pipeline.

In [3]:
z = ZipFile(ROOT / "data/raw/education/key-stage-4-performance.zip")

Find the name of the file

In [4]:
pcon_data_file = next((item for item in z.namelist() if '_sl_pcon_data_' in item), None)

Open the zip item and load into an etl source

In [5]:
with z.open(pcon_data_file) as f:
    source = etl.MemorySource(f.read())
raw_data = etl.fromcsv(source).cut(
    'time_period',
    'geographic_level',
    'pcon_code', 
    'pcon_name', 
    'avg_p8score',
    'avg_att8',
    'pt_l2basics_94',
    'pt_l2basics_95',
).selecteq(
    'geographic_level', 'Parliamentary constituency'
).selecteq(
    'time_period', '202223'
).cutout(
    'time_period', 'geographic_level'
)

In [6]:
raw_data

pcon_code,pcon_name,avg_p8score,avg_att8,pt_l2basics_94,pt_l2basics_95
E14000530,Aldershot,-0.63,41,56.7,36.1
E14000531,Aldridge-Brownhills,-0.36,42.5,58.4,36
E14000532,Altrincham and Sale West,0.4,59.6,83,67.6
E14000533,Amber Valley,-0.14,43.6,60.2,39.1
E14000534,Arundel and South Downs,0.08,47.9,68,46.5


In [7]:
raw_data.tocsv(OUT_PATH / 'attainment_scores_pcon_2010.csv')